In [6]:
import pandas as pd
import nltk
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer(text):
    return [tok for tok in j_t.tokenize(text, wakati = True)]

In [7]:
data = pd.read_csv("result_seq2seq.csv")

data

In [8]:
ans = data["answer"]
pred = data["predict"]

In [11]:
def evaluate(path):
    data = pd.read_csv(path)
    ans = data["answer"]
    pred = data["predict"]
    cnt = 0
    # 一致率
    for i in range(len(ans)):
        if ans[i] == pred[i]:
            cnt+=1
    same_percentage = (cnt/len(ans))*100
    # 種類数
    col = []
    for i in range(len(pred)):
        if pred[i] not in col:
            col.append(pred[i])
    ans_kind = len(col)
    # bleu
    df_ans = data.assign(bleu = 0)
    df_ans = df_ans.groupby("input").agg({
    'answer':list,
    "predict": list,
    "bleu" : list
    })
    df_ans = df_ans.reset_index()
    col, ans, pred = [], [], []
    for k in range(len(df_ans)):
        for j in range(len(df_ans["predict"][k])):

            for i in range(len(df_ans["answer"][k])):
                ans_str = df_ans["answer"][k][i]
                ans.append(tokenizer(ans_str))

            #print(k)
            pred = df_ans["predict"][k][j]
            #print(pred)
            pred = tokenizer(pred)
            col.append(nltk.translate.bleu_score.sentence_bleu(ans, pred))
            ave = sum(col)/len(col)
            #print(ave)
            df_ans["bleu"][k][j] = ave
            col, ans = [], []
    input_str, answer, predict, bleu = [], [], [], []
    for i in range(len(df_ans)):
        for j in range(len(df_ans["answer"][i])):
            input_str.append(df_ans["input"][i])
            answer.append(df_ans["answer"][i][j])
            predict.append(df_ans["predict"][i][j])
            bleu.append(df_ans["bleu"][i][j])
            
    df = pd.DataFrame({"input":input_str,"answer":answer,"predict":predict,"bleu":bleu})
    path = path.replace("result", "score")
    df.to_csv(path)
    blue = df["bleu"]
    l = blue.values.tolist()
    ave_blue = sum(l)/len(l)
    return same_percentage,ans_kind, ave_blue  

In [ ]:
evaluate("result_seq2seq.csv")